In [1]:
data_var = '2024-01-03'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
9030,2024-01-03,Eua Nba,00:00,Golden State Warriors,Orlando Magic,1.67,2.41,230.5,1.80,2.08,-4.5,2.09,1.85,AsYA9BBo,0.598802,0.414938,0.555556,0.480769,0.013740,0.8,0.2,NaN,NaN,202.132,91.202334,0.451202,223.210,63.741873,0.285569,208.072,307.328,180.56,321.00,0.0,0.0,0.0,0.0,0.256500,0.102057,0.086145,0.48,0.096,6.979167,0.646947,0.7,0.053053,-1.58,-0.316,-4.462025,0.415289,0.4,-0.015289
9031,2024-01-03,Eua Nba,00:00,Sacramento Kings,Charlotte Hornets,1.07,12.60,231.5,1.86,2.07,-16.5,2.06,1.87,lYWE8VQi,0.934579,0.079365,0.537634,0.483092,0.013945,0.4,0.6,NaN,NaN,197.852,42.514841,0.214882,771.456,275.438537,0.357037,174.738,808.316,165.62,762.60,0.0,1.0,0.0,0.0,1.192822,0.075569,0.068372,-1.01,-0.202,-0.346535,0.623411,0.7,0.076589,-5.00,-1.000,-11.600000,0.205580,0.1,-0.105580
9032,2024-01-03,Eua Nba,21:00,Cleveland Cavaliers,Washington Wizards,1.28,4.33,238.5,1.82,1.98,-10.5,2.07,1.71,reWI7ktb,0.781250,0.230947,0.549451,0.505051,0.012197,0.6,0.6,NaN,NaN,245.514,56.930546,0.231883,439.884,146.684231,0.333461,227.514,728.590,313.02,691.48,0.0,1.0,0.0,0.0,0.768868,0.059546,0.134687,-0.30,-0.060,-4.666667,0.589640,0.7,0.110360,-2.38,-0.476,-6.995798,0.210138,0.2,-0.010138
9033,2024-01-03,Eua Nba,21:00,Indiana Pacers,Milwaukee Bucks,2.40,1.67,257.5,1.82,1.99,1.5,2.06,1.76,dbzN69e4,0.416667,0.598802,0.549451,0.502513,0.015469,0.6,0.8,NaN,NaN,273.672,74.009181,0.270430,202.970,34.490528,0.169929,180.904,200.258,218.40,179.69,0.0,0.0,0.0,0.0,0.253655,0.063101,0.111064,-0.94,-0.188,-7.446809,0.656469,0.7,0.043531,-0.02,-0.004,-167.500000,0.617087,0.6,-0.017087
9034,2024-01-03,Eua Nba,21:30,Atlanta Hawks,Oklahoma City Thunder,1.97,1.97,248.5,1.86,1.94,1.5,1.97,1.97,vNPR5TAA,0.507614,0.507614,0.537634,0.515464,0.015228,0.6,0.4,NaN,NaN,238.066,49.308358,0.207121,216.668,43.632180,0.201378,227.314,253.202,212.30,348.10,0.0,0.0,0.0,0.0,0.000000,0.029773,0.000000,-3.43,-0.686,-1.413994,0.579140,0.3,-0.279140,-0.35,-0.070,-13.857143,0.509912,0.6,0.090088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9142,2024-01-03,Eua Ncaa,23:00,Idaho,St. Thomas (Minn.),2.61,1.51,131.5,1.91,1.80,3.5,1.83,1.83,xjebd3P7,0.383142,0.662252,0.523560,0.555556,0.045393,0.0,0.2,NaN,NaN,333.176,354.299232,1.063400,579.650,1045.124386,1.803027,0.000,593.694,101.87,140.00,0.0,0.0,0.0,0.0,0.377581,0.041931,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000,-1.41,-0.282,-1.808511,0.000000,0.0,0.000000
9143,2024-01-03,Eua Ncaa,23:30,Denver,Idaho State,1.23,3.97,151.5,1.80,1.91,-8.5,1.83,1.83,hbwoK6HE,0.813008,0.251889,0.555556,0.523560,0.064897,0.0,0.8,NaN,NaN,304.768,389.554009,1.278199,193.850,58.027913,0.299344,0.000,214.064,152.22,198.36,0.0,0.0,0.0,0.0,0.745182,0.041931,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000,-2.53,-0.506,-5.869565,0.000000,0.0,0.000000
9144,2024-01-03,Eua Ncaa,23:30,Montana State,Oral Roberts,1.90,1.90,142.5,1.91,1.80,-1.5,2.00,1.73,4xcjbPfe,0.526316,0.526316,0.523560,0.555556,0.052632,0.0,0.6,NaN,NaN,290.792,282.071075,0.970010,236.810,193.681951,0.817879,0.000,416.700,143.29,186.90,0.0,0.0,0.0,0.0,0.000000,0.041931,0.102369,0.00,0.000,inf,0.000000,0.0,0.000000,-2.90,-0.580,-1.551724,0.000000,0.0,0.000000
9145,2024-01-03,Finlândia Korisliiga,13:30,Kataja,Karhu Basket,2.34,1.57,163.5,1.80,1.92,2.5,1.99,1.73,zHJ

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:00,Eua Nba,Golden State Warriors,Orlando Magic,230.5,1.80,0.9999,Over
1,00:00,Eua Nba,Sacramento Kings,Charlotte Hornets,231.5,1.86,1.0000,Over
2,21:30,Eua Nba,Atlanta Hawks,Oklahoma City Thunder,248.5,1.86,0.9991,Over
3,16:30,Europa Euroliga,Baskonia,Panathinaikos,164.5,1.91,0.7272,Over
